In [1]:
import matplotlib.pyplot as plt 
import pandas as pd
from pyarrow import parquet 
import numpy as np
import sys 
sys.path.append('/home/bekah/gPhoton2')
from gPhoton.pipeline import execute_pipeline
from astropy.io import fits

In [6]:
low_eclipse = 2505
high_eclipse = 2507
metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','>',low_eclipse),('eclipse','<',high_eclipse),
                                           ('fuv_temp','>',0),
                                          ('legs','==',0)]).to_pandas()


In [7]:
metadata

,eclipse,obstype,legs,ra_max,ra_min,dec_max,dec_min,fuv_temp
0,2506,MIS,0,32.934511,32.907482,14.282081,14.256885,29.924175


In [ ]:
low_eclipse = 2505
high_eclipse = 2507
metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','>',low_eclipse),('eclipse','<',high_eclipse),
                                           ('fuv_temp','>',0),
                                          ('legs','==',0)]).to_pandas()


In [2]:
 execute_pipeline(23456,
                        "FUV",
                        depth=None,
                        threads=4,
                        verbose=10,
                        local_root=f"/media/bekah/BekahA/eclipses",
                        recreate=False,
                        aperture_sizes=[12.8],
                        write={"movie": False, "image": True},
                        compression="rice",
                        lil=True)

starting timer
eclipse 23456 FUV  -- MIS; 0 leg(s)
downloading raw6file
Processing eclipse 23456
28.7778329060574
Offsetting FUV image for eclipse 23456 at 28.7778329060574 degrees.
fuvtemp: 28.7778329060574
Setting FUV offsets to x=-0.12211, y=0.22794350369115263
fodx_coef_0: -0.12211, fody_coef_0: 0.14803
xoffset: -0.12211, yoffset: 0.22794350369115263
trange= ( 874361241.995 , 874362946.995 )                   
RA AVG: 323.48945087374165, DEC AVG: -2.066535934111889, ROLL AVG: 23.639969174886176
Loading raw6 file...                                        
4035551 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = -380625.36080072116, 0.0004411392059765214
[8.74361652e+08 8.74361652e+08 8.74361652e+08 ... 8.74362085e+08
 8.74362085e+08 8.74362085e+08][8.74362085e+08 8.74362085e+08 8.74362085e+08 ... 8.74362519e+08
 8.74362519e+08 8.74362519e+08][8.74361242e+08 8.74361242e+08 8.74361242e+08 ... 8.74361652e+08
 8.74361652e+08 8.74361652e+08]




'return code: successful'

In [ ]:
eclipse_list = [922, 1910, 2117, 2188, 2313, 2344, 2372, 2461, 2477,
              2492, 2506, 2564, 2618]
def make_eclipses(): 
    bands = ["NUV","FUV"]
    for eclipse in eclipse_list: 
        #eclipse = str(eclipse).zfill(5)
        print(eclipse)
        for band in bands: 
            try:
                 execute_pipeline(
                        eclipse,
                        band,
                        depth=None,
                        threads=4,
                        verbose=10,
                        local_root=f"/media/bekah/BekahA/eclipses",
                        recreate=False,
                        aperture_sizes=[12.8],
                        write={"movie": False, "image": True},
                        compression="rice",
                        lil=True)
            except: 
                print("no work :( ")

In [ ]:
make_eclipses()

In [ ]:
short_list = eclipse_nums[0::10]
len(short_list)


In [ ]:
# looking at offset between sources for a single eclipse 
# use dbscan to group points based on distance 

eclipse = 23154

def get_selected_sources(eclipse):
    """ get central sources from both fuv and nuv for the same eclipse. returns concat
    pd df with sources from both bands. """
    
    list_folder_path = "/media/bekah/BekahA/eclipses/"
    eclipse_folder_path = "/media/bekah/BekahA/eclipses/e"+str(eclipse)+"/"
    nuv_list_name = f"e{eclipse}, FUV, sources_pts.csv"
    fuv_list_name = f"e{eclipse}, NUV, sources_pts.csv"
    nuv_image_name = f"e{eclipse}-nd-full-0-rice.fits"
    fuv_image_name = f"e{eclipse}-fd-full-0-rice.fits"

    nuvexists, nuvpath = check_file_in_folder(list_folder_path, nuv_list_name) 
    fuvexists, fuvpath = check_file_in_folder(list_folder_path, fuv_list_name) 

    if nuvexists & fuvexists: 
        nuv = pd.read_csv(nuvpath)  
        nuv['band'] = 'nuv'
        fuv = pd.read_csv(fuvpath) 
        fuv['band'] = 'fuv'

        # filter for central sources
        selected_nuv = nuv[(nuv['xcentroid'] > 1000) & (nuv['xcentroid'] < 2000) &
                           (nuv['ycentroid'] > 1000) & (nuv['ycentroid'] < 2000)]
        selected_fuv = fuv[(fuv['xcentroid'] > 1000) & (fuv['xcentroid'] < 2000) &
                           (fuv['ycentroid'] > 1000) & (fuv['ycentroid'] < 2000)]

        fuvandnuv = pd.concat([selected_nuv, selected_fuv])

        return fuvandnuv 

    # grouping 
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

def dbscan_group(fuvandnuv): 
    fuvandnuv_xy = list(zip(fuvandnuv['xcentroid'],fuvandnuv['ycentroid']))
    dbscan = DBSCAN(eps=10, min_samples=2)
    labels = dbscan.fit_predict(fuvandnuv_xy)
    points_with_labels = zip(fuvandnuv_xy, labels)
    
    labelsdf = pd.DataFrame(points_with_labels, columns=['Point', 'Label'])
    labelsdf[['X', 'Y']] = labelsdf['Point'].apply(lambda x: pd.Series({'X': x[0], 'Y': x[1]}))
    labelsdf = labelsdf.drop('Point', axis=1)
    return labelsdf 

In [ ]:
# plot dbscan groups 
labeled_points = dbscan_group(fuvandnuv)
%matplotlib notebook
plt.scatter(labeled_points['X'],labeled_points['Y'],c=labeled_points['Label'])
plt.colorbar()

In [ ]:
# plot star locations for fuv and nuv 
plt.scatter(selected_fuv['xcentroid'],selected_fuv['ycentroid'])
plt.scatter(selected_nuv['xcentroid'],selected_nuv['ycentroid'],c='red')

In [ ]:
# try using the segmentation images, which should be more accurate than just centroids

eclipse_folder_path = "/media/bekah/BekahA/eclipses/e"+str(eclipse)+"/"
nuv_seg_image = f"e{eclipse}-nd-segmentation_leg0.jpg"
fuv_seg_image = f"e{eclipse}-fd-segmentation_leg0.jpg"

nuvexists, nuvpath = check_file_in_folder(eclipse_folder_path, nuv_seg_image) 
fuvexists, fuvpath = check_file_in_folder(eclipse_folder_path, fuv_seg_image) 

plt.imshow(nuvpath)

#RESULT: seg images are dtype <U67  and therefore mb not high quality enough for this 

In [ ]:
# try using star locations + radius to draw shapes akin to seg map?
# for every point in fuv, find it's closest match in nuv?

sources = get_selected_sources(eclipse)

In [ ]:
fuv_dx = "/home/bekah/glcat/fuv_offset/fuv_dx_fdttdc_coef_0.tbl.txt"
fuv_dy = "/home/bekah/glcat/fuv_offset/fuv_dy_fdttdc_coef_0.tbl.txt"

fuv_dx_df = pd.read_csv(fuv_dx, delim_whitespace=True, skiprows=0)
fuv_dy_df = pd.read_csv(fuv_dy, delim_whitespace=True, skiprows=0)
fuv_dx_df = fuv_dx_df.drop(columns=['coef_0', '|.2','|.1'])
fuv_dx_df = fuv_dx_df.rename(columns={"|": "ecl", "ecl": "coef_0"})
fuv_dy_df = fuv_dy_df.drop(columns=['coef_0', '|.2','|.1'])
fuv_dy_df = fuv_dy_df.rename(columns={"|": "ecl", "ecl": "coef_0"})

In [ ]:
plt.clf()
plt.scatter(fuv_dy_df['ecl'],fuv_dy_df['coef_0'])
plt.xlabel('eclipse')
plt.ylabel('fuv dy coef 0')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

ax1.scatter(fuv_dx_df['ecl'],fuv_dx_df['coef_0'], color='blue', label='dx coef 0')
ax1.set_ylabel('fuv dx coef 0')

ax2.scatter(fuv_dx_df['ecl'],fuv_dy_df['coef_0'], color='orange', label='dy coef 0')
ax2.set_xlabel('eclipse')
ax2.set_ylabel('fuv dy coef 0')

ax1.legend()
ax2.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
 
    
# DY CHANGE POINTS 

# slopes between consecutive points
slopes = np.diff(fuv_dy_df['coef_0']) / np.diff(fuv_dy_df['ecl'])

dy_change_points = np.where(np.diff(np.sign(slopes)))[0] + 1

plt.scatter(fuv_dy_df['ecl'], fuv_dy_df['coef_0'])

# highlight points where slope changes
plt.scatter(fuv_dy_df['ecl'][dy_change_points], fuv_dy_df['coef_0'][dy_change_points], color='red', label='dy Slope Change Points')

plt.xlabel('eclipse')
plt.ylabel('fuv dy coef 0')
plt.legend()
plt.show()

In [ ]:

#DX CHANGE POINTS 

# slopes between consecutive points
slopes = np.diff(fuv_dx_df['coef_0']) / np.diff(fuv_dx_df['ecl'])

dx_change_points = np.where(np.diff(np.sign(slopes)))[0] + 1

plt.scatter(fuv_dx_df['ecl'], fuv_dx_df['coef_0'])

# highlight points where slope changes
plt.scatter(fuv_dx_df['ecl'][dx_change_points], fuv_dx_df['coef_0'][dx_change_points], color='red', label='dx Slope Change Points')

plt.xlabel('eclipse')
plt.ylabel('fuv dx coef 0')
plt.legend()
plt.show()

In [ ]:
print(f"dx change points: {len(dx_change_points)}")
print(f"dy change points: {len(dy_change_points)}")

In [ ]:
plt.clf()

fig, ax = plt.subplots()

ax.scatter(fuv_dx_df['ecl'],fuv_dx_df['coef_0'],s=.5, label= 'dx coef 0')
ax.scatter(fuv_dy_df['ecl'],fuv_dy_df['coef_0'],c='red',s=.5,label= 'dy coef 0')
plt.xlabel('eclipse')
plt.ylabel('coef 0')

for line_x in dx_change_points:
    plt.axvline(x=line_x, color='lightblue', linestyle='--',lw=.5, label=line_x)
    
for line_x in dy_change_points:
    plt.axvline(x=line_x, color='lightcoral', linestyle='--',lw=.5, label=line_x)
    
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncol=2)

In [ ]:
dx_change_points

In [ ]:
dx_points_df = pd.DataFrame()
for eclipse in dx_change_points: 
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse)]).to_pandas()
    dx_points_df = pd.concat([dx_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse+1)]).to_pandas()
    dx_points_df = pd.concat([dx_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse+2)]).to_pandas()
    dx_points_df = pd.concat([dx_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse-1)]).to_pandas()
    dx_points_df = pd.concat([dx_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse-2)]).to_pandas()
    dx_points_df = pd.concat([dx_points_df, metadata], ignore_index=True)

dx_points_df

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(dx_points_df)

In [ ]:
plt.scatter(dx_points_df['ra_max'],dx_points_df['dec_max'],c=dx_points_df['eclipse'],s=.9)

In [ ]:
dy_points_df = pd.DataFrame()
for eclipse in dy_change_points: 
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse)]).to_pandas()
    dy_points_df = pd.concat([dy_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse+1)]).to_pandas()
    dy_points_df = pd.concat([dy_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse+2)]).to_pandas()
    dy_points_df = pd.concat([dy_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse-1)]).to_pandas()
    dy_points_df = pd.concat([dy_points_df, metadata], ignore_index=True)
    metadata = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                                  filters=[('eclipse','==',eclipse-2)]).to_pandas()
    dy_points_df = pd.concat([dy_points_df, metadata], ignore_index=True)


color_mapping = {'DIS': 'red', 'AIS': 'blue', 'MIS': 'lightblue', 'GII': 'orange', 'CAI': 'hotpink', 
                'unknown': 'cadetblue'}

dy_points_df['Color'] = dy_points_df['obstype'].map(color_mapping)


In [ ]:
plt.scatter(dy_points_df['eclipse'],dy_points_df['fuv_temp'],c=dy_points_df["Color"])
plt.xlabel('eclipse')
plt.ylabel('fuv_temp')

In [ ]:
# plotting eclipses in ra, dec space with eclipse number 
plt.scatter(dy_points_df['ra_max'],dy_points_df['dec_max'],c=dy_points_df["eclipse"], s=5)
for i in range(len(dy_points_df)):
    plt.text(dy_points_df['ra_max'].iloc[i], 
             dy_points_df['dec_max'].iloc[i], 
             dy_points_df['eclipse'].iloc[i],
             va='bottom', ha='center',
            fontsize=6)
plt.xlabel('ra')
plt.ylabel('dec')
plt.show()

In [ ]:
dx_differences = [second - first for first, second in zip(dx_change_points, dx_change_points[1:])]
dy_differences = [second - first for first, second in zip(dy_change_points, dy_change_points[1:])]
dy_differences

In [ ]:
dx_differences

In [ ]:
from gPhoton.io._query.mast_query import retrieve_aspect
from gPhoton.io.mast import get_raw_paths

get_raw_paths(19261)

In [ ]:
scst_hdu = fits.open("/home/bekah/glcat/fuv_offset/e19261-scst.fits")

In [ ]:
fdttdc = scst_hdu[1].data['FDTTDC'].byteswap().newbyteorder() 
pktime = scst_hdu[1].data['pktime'].byteswap().newbyteorder() 
hvnom_fuv = scst_hdu[1].data['hvnom_fuv'].byteswap().newbyteorder() 
t_dead_fuv = scst_hdu[1].data['t_dead_fuv'].byteswap().newbyteorder() 
fdtdet = scst_hdu[1].data['FDTDET'].byteswap().newbyteorder() 

In [ ]:
scst_df = pd.DataFrame({'pktime': pktime, 'fdttdc': fdttdc,
                        'hvnom_fuv': hvnom_fuv, 't_dead_fuv': t_dead_fuv,
                       'fdtdet': fdtdet})

#trange= ( 849542598.995 , 849544054.995 )           
min_time = float(849542598.995)
max_time = float(849544054.995)
subset_scst_df = scst_df[(scst_df['pktime'] >= min_time) & (scst_df['pktime'] <= max_time)]


In [ ]:
subset_scst = pd.read_csv("/home/bekah/glcat/fuv_offset/subset_scst_df.csv")

In [ ]:
plt.clf()

plt.scatter(subset_scst['pktime'],subset_scst['fdttdc'])
plt.xlabel('pktime')
plt.ylabel("FDTTDC")

plt.show()

In [ ]:
plt.hist(subset_scst['fdttdc'])
plt.xlabel("fdttdc")

In [ ]:
plt.scatter(subset_scst_df['pktime'],subset_scst_df['fdttdc'])

In [ ]:
plt.scatter(hvnom_fuv, fdttdc)
plt.xlabel('hvnom_fuv')
plt.ylabel("FDTTDC")
plt.title("eclipse 19261 scst file data")

In [ ]:
# calculate the slope change across eclipse time if all fdttdc data were used 

result = -0.07478216490196762 # in gPhoton2 
# from lookup table:     
fody_coef_0 = -0.03934
fody_coef_1 = 0.3597
fuv_temp = 29.0985325685348

# gPhoton2 check 
yoffset = fody_coef_0 - (fody_coef_1 * (fuv_temp - 29.0))

print(yoffset)
print(f"diff: {yoffset-result}")

# ok great so I can recreate what gPhoton2 does 
# gPhoton2 uses -0.07478216490196762 as the dy correction for all of 19261 

In [ ]:
yoffset_col = fody_coef_0 - (fody_coef_1 * (subset_scst['fdttdc'] - 29.0))
yoffset_col

In [ ]:
plt.hist(yoffset_col)
plt.axvline(x=yoffset, color='red', linestyle='--',lw=1, label="gPhoton2 y offset")
plt.xlabel("ecl 19261: y offset with variable fdttdc (arcseconds)")
plt.show()

In [ ]:
# ok let's do all of the above again for eclipse 3801 
from gPhoton.io.mast import get_raw_paths

paths = get_raw_paths(3801)

In [ ]:
# run gPhoton2 on eclipse
execute_pipeline(
                3801,
                "FUV",
                depth=None,
                threads=4,
                verbose=10,
                local_root=f"/media/bekah/BekahA/eclipses",
                recreate=False,
                aperture_sizes=[12.8],
                write={"movie": False, "image": True},
                compression="rice",
                lil=True)

In [ ]:
# calculate the slope change across eclipse time if all fdttdc data were used 

result = -0.4705166976569246 # in gPhoton2 
# from lookup table:     
fody_coef_0 = -0.25999
fody_coef_1 = 0.3597
fuv_temp = 29.5852841191463

# gPhoton2 check 
yoffset = fody_coef_0 - (fody_coef_1 * (fuv_temp - 29.0))

print(yoffset)
print(f"diff: {yoffset-result}")

# ok great so I can recreate what gPhoton2 does 
# gPhoton2 uses -0.07478216490196762 as the dy correction for all of 19261 

In [ ]:
# make df 
scst_path = "/home/bekah/glcat/fuv_offset/e03801-scst.fits"
eclipse = 3801
# trange= ( 758062104.995 , 758063802.995 )                   
min_time = float(758062104.995)
max_time = float(758063802.995 ) 
save_csv = f"/home/bekah/glcat/fuv_offset/subset_scst_{eclipse}.csv"

scst_hdu = fits.open(scst_path)

fdttdc = scst_hdu[1].data['FDTTDC'].byteswap().newbyteorder() 
pktime = scst_hdu[1].data['pktime'].byteswap().newbyteorder() 
hvnom_fuv = scst_hdu[1].data['hvnom_fuv'].byteswap().newbyteorder() 
t_dead_fuv = scst_hdu[1].data['t_dead_fuv'].byteswap().newbyteorder() 
fdtdet = scst_hdu[1].data['FDTDET'].byteswap().newbyteorder() 

scst_df = pd.DataFrame({'pktime': pktime, 'fdttdc': fdttdc,
                        'hvnom_fuv': hvnom_fuv, 't_dead_fuv': t_dead_fuv,
                       'fdtdet': fdtdet})


subset_scst_df = scst_df[(scst_df['pktime'] >= min_time) & (scst_df['pktime'] <= max_time)]

subset_scst_df.to_csv(save_csv)

In [ ]:
subset_scst = pd.read_csv("/home/bekah/glcat/fuv_offset/subset_scst_3801.csv")
yoffset_col_3801 = fody_coef_0 - (fody_coef_1 * (subset_scst['fdttdc'] - 29.0))
yoffset_col_3801


In [ ]:
plt.hist(yoffset_col_3801)
plt.axvline(x=yoffset, color='red', linestyle='--',lw=1, label="gPhoton2 y offset")
plt.xlabel("ecl 3801: y offset with variable fdttdc (arcseconds)")